In [20]:
%load_ext autoreload
%autoreload 2

In [7]:
from aggregate_cross_validation_data import *
import pandas as pd

In [8]:
# aurocs, spearmans, mses, maes = aggregate_cross_val_data(
#     "./training_results_v2/v7_cell_models_augmented_2_epochs/cellmodel_prior-none_cellreg-enabled_heldout-{patient}"
# )

# print("*** CELL MODELS ***")
# print("spearman:", bootstrap(spearmans)['formatted'])
# print("auroc:", bootstrap(aurocs)['formatted'])
# print("mse:", bootstrap(mses)['formatted'])
# print("mae:", bootstrap(maes)['formatted'])

# aurocs, spearmans, mses, maes = aggregate_cross_val_data(
#     "./training_results_v2/v6_cnn_models_augmented_2_epochs/inception-holding-out-{patient}"
# )

# print("*** INCEPTION MODELS ***")
# print("spearman:", bootstrap(spearmans)['formatted'])
# print("auroc:", bootstrap(aurocs)['formatted'])
# print("mse:", bootstrap(mses)['formatted'])
# print("mae:", bootstrap(maes)['formatted'])

genes = open("training_results_v2/v2_repr_contrastive/genes.txt").read().split("\n")
dataframes = {}
for prior in ['none', 'repr']:
    for cre in [False, True]:
        cre_str = 'cellreg-enabled_' if cre else ''
        aurocs, spearmans, mses, maes = aggregate_cross_val_data(
            f"./training_results_v2/v2_repr_contrastive/cellmodel_prior-{prior}_{cre_str}heldout-{{patient}}"
        )
        cre_str2 = 'CellRegularized' if cre else 'NonCellRegularized'
        prior_str = 'Unpretrained' if prior == 'none' else 'ContrastivePretrained'
        out_file = f'crossval_{cre_str2}_{prior}.csv'
        df = pd.DataFrame({
            "AUROC": list(aurocs),
            "Spearman": list(spearmans),
            "MSE": list(mses),
            "MAE": list(maes),
        }, index=genes)
        df.index.name = 'Gene'
        df.to_csv(out_file)
        dataframes[cre_str2 + "_" + prior_str] = df
        print(out_file)
        print("spearman:", bootstrap(spearmans)['formatted'])
        print("auroc:", bootstrap(aurocs)['formatted'])
        print("mse:", bootstrap(mses)['formatted'])
        print("rmse:", bootstrap(mses ** 0.5)['formatted'])
        print("mae:", bootstrap(maes)['formatted'])


Skipping ./training_results_v2/v2_repr_contrastive/cellmodel_prior-none_heldout-092842_16 due to [Errno 2] No such file or directory: './training_results_v2/v2_repr_contrastive/cellmodel_prior-none_heldout-092842_16_stats.json'
crossval_NonCellRegularized_none.csv
spearman: 0.5591 ± 0.0146
auroc: 0.8093 ± 0.0083
mse: 0.5164 ± 0.0163
rmse: 0.7186 ± 0.0113
mae: 0.5559 ± 0.0122
crossval_CellRegularized_none.csv
spearman: 0.5381 ± 0.0158
auroc: 0.8012 ± 0.0074
mse: 0.6194 ± 0.0220
rmse: 0.7870 ± 0.0140
mae: 0.6152 ± 0.0119
crossval_NonCellRegularized_repr.csv
spearman: 0.5481 ± 0.0166
auroc: 0.8094 ± 0.0087
mse: 0.5713 ± 0.0175
rmse: 0.7559 ± 0.0116
mae: 0.5946 ± 0.0109
crossval_CellRegularized_repr.csv
spearman: 0.5366 ± 0.0168
auroc: 0.8049 ± 0.0093
mse: 0.7020 ± 0.0278
rmse: 0.8378 ± 0.0166
mae: 0.6428 ± 0.0134


In [14]:
inception_example_validation = torch.load("./training_results_v2/v6_cnn_models_augmented_2_epochs/inception-holding-out-autostainer_validation_results.pt", map_location='cpu')

In [19]:
inception_example_validation.keys()

dict_keys(['predictions', 'true_log_counts', 'stats', 'stats_bootstrapped'])

In [21]:
def aggregate_cross_val_data_inception(model_identifier_format: str):
    variant_aurocs = []
    variant_spearmans = []
    variant_mses = []
    variant_maes = []

    # take mean of each stat for each gene
    for patient in patients_to_hold_out:
        model_identifier = model_identifier_format.format(patient=patient)

        try:
            # load stats
            with open(model_identifier + "_stats.json") as f:
                stats = json.load(f)['stats']

            variant_aurocs.append(stats['auroc_values'])
            variant_spearmans.append(stats['spearman_values'])
            variant_mses.append(stats['mse_values'])

            results = torch.load(model_identifier + "_validation_results.pt", map_location='cpu')
            preds = results['predictions']
            trues = results['true_log_counts']
            variant_maes.append(torch.abs(preds - trues).mean(dim=0).cpu().numpy())

        except Exception as e:
            print("Skipping " + model_identifier + " due to " + str(e))
    
    return (
        np.array(variant_aurocs).mean(axis=0),
        np.array(variant_spearmans).mean(axis=0),
        np.array(variant_mses).mean(axis=0),
        np.array(variant_maes).mean(axis=0),
    )

In [30]:
# for inception
aurocs, spearmans, mses, maes = aggregate_cross_val_data_inception(
    f"./training_results_v2/v6_cnn_models_augmented_2_epochs/inception-holding-out-{{patient}}"
)

df = pd.DataFrame({
    "AUROC": list(aurocs),
    "Spearman": list(spearmans),
    "MSE": list(mses),
    "MAE": list(maes),
}, index=genes)
df.index.name = 'Gene'
df.to_csv("crossval_inception.csv")

dataframes['inception'] = df

In [29]:
np.median(spearmans)

0.5723838673036825

In [31]:
import pickle

with open("cv_dataframes.pkl", "wb") as f:
    pickle.dump(dataframes, f)

In [37]:
dataframes.keys()

dict_keys(['NonCellRegularized_Unpretrained', 'CellRegularized_Unpretrained', 'NonCellRegularized_ContrastivePretrained', 'CellRegularized_ContrastivePretrained', 'inception'])

In [38]:
# top 10% of genes from each approach

print(
    '\n'.join(dataframes['inception'].sort_values("AUROC").tail(100).index.tolist())
)


HSP90AB1
C19orf33
CANT1
TRAF4
TMEM238
TMEM141
STK39
TSPAN13
MT-ND2
ATP1A1
MYO6
ITM2C
CDH1
TMPRSS2
RHPN2
MT-CYB
B3GNT3
MT-ATP6
TSPAN1
MGAT4B
SFN
MT-CO2
CGN
MAPK13
RAB11FIP1
PRSS3
S100A10
TMEM106C
SRPK1
ITGA6
PPP1R1B
FAM83H
PYGB
MAL2
CAMK2N1
CLDN3
MT-CO3
MYO5B
ANXA4
PATJ
TSPAN3
TMEM54
PLS1
SERPINB1
MT-ND4L
SYNGR2
HNF4A
PKP3
CD9
INAVA
S100A14
TMPRSS4
EHF
LSR
DNM2
MT-ND3
LLGL2
MT-ND4
BAIAP2L1
GMDS
SPINT1
ESRP1
EPS8L3
USH1C
MT-ND5
GOLM1
RBM47
EZR
LGALS3
LGALS3BP
POF1B
RAB25
TSPAN8
MT-ND1
SLC44A4
DSG2
KRT19
ATP1B1
FXYD3
LAD1
JUP
STARD10
MUC13
CD2AP
MISP
CD24
PRR15L
MT-ND6
ST14
KLF5
TPD52
CLDN4
DSP
KRT8
SPINT2
EPCAM
LGALS4
MYH14
CLDN7
ELF3


In [39]:
print(
    '\n'.join(dataframes['NonCellRegularized_Unpretrained'].sort_values("AUROC").tail(100).index.tolist())
)

MLEC
PERP
TMPRSS2
ESRP2
CANT1
MGAT4B
MAL2
STK39
PYGB
FERMT1
AKAP1
MAP7
CAMK2N1
ITGB4
S100A10
TMEM54
ABHD17C
BAIAP2L1
ATP1A1
PPP1R1B
CDH1
LSR
TSPAN8
ACSL5
ERBB3
HSP90AA1
ITGA6
B3GNT3
MYO6
HNF4A
TMEM106C
FXYD3
TSPAN3
PLS1
MYO5B
SLC44A4
MAPK13
PATJ
MARCKSL1
KRT19
FAM83H
ANXA4
PKP3
MT-ATP6
DNM2
LGALS3
SPINT1
SRPK1
TMPRSS4
EHF
RAB25
MT-CO2
EZR
ESRP1
ATP2C2
CD9
CLDN3
LLGL2
MT-ND2
POF1B
RBM47
GMDS
EPS8L3
TMSB4X
ATP1B1
DSG2
USH1C
SERPINB1
GOLM1
TMSB10
SYNGR2
MT-ND3
MUC13
MT-ND4L
LAD1
MT-ND4
LGALS3BP
MT-CYB
MT-CO3
CLDN4
CD24
MT-ND5
CD2AP
MT-ND1
JUP
ST14
PRR15L
MISP
STARD10
LGALS4
EPCAM
KLF5
MT-ND6
KRT8
SPINT2
TPD52
MYH14
DSP
ELF3
CLDN7


In [40]:
print(
    '\n'.join(dataframes['CellRegularized_Unpretrained'].sort_values("AUROC").tail(100).index.tolist())
)

RNF128
GRHL2
GPX2
TRAF4
ERBB3
MAL2
MGAT4B
SCNN1A
CAMK2N1
MARCKSL1
CDH1
TMEM141
STK39
DDR1
PRSS3
INAVA
MAP7
TMPRSS2
FERMT1
DNM2
TSPAN3
ACSL5
MYO6
ATP1A1
VIL1
TSPAN13
SYNGR2
MYO5B
RHPN2
FXYD3
ATP2C2
TMSB10
PLS1
SERPINB1
LSR
FAM83H
CLDN3
MAPK13
PPP1R1B
ITGA6
KRT19
RBM47
ANXA4
PATJ
S100A14
TMEM106C
EZR
HNF4A
SRPK1
LGALS3
EPS8L3
SPINT1
LLGL2
CD9
LGALS3BP
MT-CO3
PKP3
TMPRSS4
BAIAP2L1
MT-ATP6
MT-ND4
SLC44A4
MT-CO2
MT-ND2
GOLM1
MUC13
EHF
POF1B
MT-ND4L
USH1C
TSPAN8
ATP1B1
DSG2
ESRP1
GMDS
RAB25
MISP
MT-CYB
JUP
CLDN4
LAD1
CD2AP
CD24
STARD10
ST14
MT-ND3
MT-ND5
KRT8
MT-ND1
LGALS4
MT-ND6
SPINT2
PRR15L
EPCAM
KLF5
TPD52
MYH14
CLDN7
DSP
ELF3


In [42]:
print(
    '\n'.join(dataframes['NonCellRegularized_ContrastivePretrained'].sort_values("AUROC").tail(100).index.tolist())
)

VIL1
GRHL2
ACSL5
RAB11FIP1
ITPR3
CAMK2N1
AGR2
PRSS3
INAVA
MAP7
TMEM54
DDR1
TRAF4
TMEM141
MLEC
HSP90AB1
TSPAN3
HSP90AA1
STK39
S100A10
MGAT4B
CLDN3
PLS1
MARCKSL1
MYO6
MYO5B
MAL2
MAPK13
TMSB4X
GPX2
FERMT1
EPS8L3
DNM2
TMEM106C
HNF4A
ANXA4
ITGA6
FAM83H
LSR
MUC13
S100A14
PPP1R1B
FXYD3
PATJ
RBM47
LGALS3
SPINT1
TMPRSS4
TMSB10
LLGL2
BAIAP2L1
SERPINB1
CD9
KRT19
PKP3
SYNGR2
EZR
SRPK1
EHF
TSPAN8
POF1B
GOLM1
USH1C
ATP1B1
SLC44A4
DSG2
GMDS
ESRP1
RAB25
LGALS3BP
MT-ATP6
MISP
STARD10
MT-CO3
MT-ND2
JUP
CD2AP
PRR15L
MT-CO2
CLDN4
MT-ND4
ST14
MT-ND5
MT-CYB
MT-ND3
LAD1
SPINT2
CD24
MT-ND4L
MT-ND1
LGALS4
TPD52
KRT8
MT-ND6
KLF5
EPCAM
MYH14
CLDN7
DSP
ELF3


In [43]:
print(
    '\n'.join(dataframes['CellRegularized_ContrastivePretrained'].sort_values("AUROC").tail(100).index.tolist())
)

RNF128
ACSL5
SCNN1A
ESRP2
GRHL2
STK39
HOOK1
AGR2
VIL1
MAP7
ERBB3
KRTCAP3
ANXA4
MYO6
MARCKSL1
GALNT3
LGALS3
INAVA
CDH1
SEL1L3
PRSS3
GOLM1
TMEM141
TRAF4
UGT8
FXYD3
TMEM106C
EZR
LSR
TSPAN13
TMPRSS2
CLDN3
ATP2C2
MAPK13
RBM47
BAIAP2L1
CD9
HSP90AB1
PLS1
MT-CO3
MAL2
KRT19
PATJ
LGALS3BP
MYO5B
GPX2
SYNGR2
FAM83H
SERPINB1
SPINT1
S100A14
FERMT1
HNF4A
SLC44A4
TMPRSS4
PKP3
MT-ATP6
TSPAN8
EPS8L3
ITGA6
LLGL2
MUC13
MT-CYB
MT-ND2
PPP1R1B
SRPK1
EHF
MT-ND4
CD2AP
MT-CO2
ATP1B1
GMDS
MT-ND4L
POF1B
RAB25
MT-ND5
JUP
USH1C
MISP
DSG2
MT-ND3
MT-ND6
ST14
CLDN4
KRT8
ESRP1
MT-ND1
SPINT2
LAD1
STARD10
PRR15L
MYH14
LGALS4
TPD52
EPCAM
CLDN7
KLF5
CD24
ELF3
DSP
